In [1]:
%reload_ext autoreload 
%autoreload 2

import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import random 
import time
from utils import Dataset,ProgressBar
from tflearn.data_flow import DataFlow,DataFlowStatus,FeedDictFlow
from tflearn.data_utils import Preloader,ImagePreloader
import scipy
import pandas as pd
import xmltodict
import common
import tflearn
import copy
from cchess import *
from game_convert import convert_game,convert_game_value,convert_game_board
import os, shutil
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
from net.net_maintainer import NetMatainer

/usr/local/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
!nvidia-smi

Wed May 30 22:23:54 2018       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 384.111                Driver Version: 384.111                   |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:05:00.0 Off |                  N/A |
| 23%   36C    P8    18W / 250W |    167MiB / 11172MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 108...  Off  | 00000000:42:00.0 Off |                  N/A |
| 23%   36C    P8     8W / 250W |   1353MiB / 11172MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
        

# a network predict select and move of Chinese chess, with minimal preprocessing

In [3]:
stamp = time.strftime('%Y-%m-%d_%H-%M-%S',time.localtime(time.time()))

In [4]:
data_dir = os.path.join('data/history_selfplays',stamp)
data_dir = 'data/history_selfplays/2018-05-30_18-41-03/'

In [5]:
if os.path.exists(data_dir):
    print('data_dir already exist: {}'.format(data_dir))
else:
    print('creating data_dir: {}'.format(data_dir))
    os.mkdir("{}".format(data_dir))

data_dir already exist: data/history_selfplays/2018-05-30_18-41-03/


In [6]:
GPU_CORE = [0]
BATCH_SIZE = 512
BEGINING_LR = 0.01
#TESTIMG_WIDTH = 500
model_name = 'update_model'

In [7]:
distribute_dir = 'data/distributed/'
filelist = os.listdir(distribute_dir)
filelist = os.listdir(data_dir)
#filelist = [os.path.join(distribute_dir,i) for i in filelist]

In [8]:
network_dir = 'data/prepare_weight/'

In [9]:
filelist[:5],len(filelist)

(['2018-05-30_17-18-12_432_mcts-mcts_b.cbf',
  '2018-05-30_18-33-06_236_mcts-mcts_peace.cbf',
  '2018-05-30_17-13-51_148_mcts-mcts_peace.cbf',
  '2018-05-30_16-59-31_35_mcts-mcts_b.cbf',
  '2018-05-30_16-36-55_853_mcts-mcts_w.cbf'],
 2300)

for f in filelist:
    src = os.path.join(distribute_dir,f)
    dst = os.path.join(data_dir,f)
    shutil.move(src,dst)

In [10]:
filelist = [os.path.join(data_dir,i) for i in filelist]

In [11]:
#filelist = filelist[:1000]
len(filelist)

2300

In [12]:
labels = common.board.create_uci_labels()
label2ind = dict(zip(labels,list(range(len(labels)))))

In [13]:
rev_ab = dict(zip('abcdefghi','abcdefghi'[::-1]))
rev_num = dict(zip('0123456789','0123456789'[::-1]))

In [107]:
class ElePreloader(object):
    def __init__(self,filelist,batch_size=64):
        self.batch_size=batch_size
        #content = pd.read_csv(datafile,header=None,index_col=None)
        self.filelist = filelist#[i[0] for i in content.get_values()]
        self.pos = 0
        self.feature_list = {"red":['A', 'B', 'C', 'K', 'N', 'P', 'R']
                             ,"black":['a', 'b', 'c', 'k', 'n', 'p', 'r']}
        self.batch_size = batch_size
        self.batch_iter = self.__iter()
        assert(len(self.filelist) > batch_size)
        self.game_iterlist = [None for i in self.filelist]
    
    def __iter(self):
        retx1,rety1,retx2,rety2 = [],[],[],[]
        vals = []
        filelist = []
        while True:
            for i in range(self.batch_size):
                if self.game_iterlist[i] == None:
                    if len(filelist) == 0:
                        filelist = copy.copy(self.filelist)
                        random.shuffle(filelist)
                    self.game_iterlist[i] = convert_game_value(filelist.pop(),self.feature_list,None)
                game_iter = self.game_iterlist[i]
                
                try:
                    x1,y1,val1 = game_iter.__next__()
                    x1 = np.transpose(x1,[1,2,0])
                    x1 = np.expand_dims(x1,axis=0)
                    
                    #if random.random() < 0.5:
                    #    y1 = [rev_ab[y1[0]],y1[1],rev_ab[y1[2]],y1[3]]
                    #    x1 = x1[:,:,::-1,:]
                    #    #x1 = np.concatenate((x1[:,::-1,:,7:],x1[:,::-1,:,:7]),axis=-1)
                    retx1.append(x1)
                    #rety1.append(y1)
                    oney = np.zeros(len(labels))
                    oney[label2ind[''.join(y1)]] = 1
                    rety1.append(oney)
                    vals.append(val1)

                    if len(retx1) >= self.batch_size:
                        yield (np.concatenate(retx1,axis=0),np.asarray(rety1),np.asarray(vals))
                        retx1,rety1 = [],[]
                        vals = []
                except :
                    self.game_iterlist[i] = None

    def __getitem__(self, id):
        
        x1,y1,val1 = self.batch_iter.__next__()
        return x1,y1,val1
        
    def __len__(self):
        return len(self.filelist)

In [108]:
filelist[0]

'data/history_selfplays/2018-05-30_18-41-03/2018-05-30_17-18-12_432_mcts-mcts_b.cbf'

In [109]:
trainset = ElePreloader(filelist=filelist,batch_size=BATCH_SIZE)
with tf.device("/gpu:{}".format(GPU_CORE[0])):
    coord = tf.train.Coordinator()
    trainflow = FeedDictFlow({
            'data':trainset,
        },coord,batch_size=BATCH_SIZE,shuffle=False,continuous=True,num_threads=1)
trainflow.start()

In [126]:
sample_x1,sample_y1,sample_value = trainflow.next()['data']

In [150]:
filelist[ind]

'data/history_selfplays/2018-05-30_18-41-03/2018-05-30_16-59-31_35_mcts-mcts_b.cbf'

sample_x1,sample_y1,sample_value = testflow.next()['data']

In [127]:
sample_x1.shape,sample_y1.shape,sample_value.shape

((512, 10, 9, 14), (512, 2086), (512,))

In [128]:
labels[np.argmax(sample_y1[0])]

'i0h0'

In [129]:
np.sum(sample_x1[0],axis=-1)

array([[1, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0],
       [1, 0, 1, 0, 1, 0, 1, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 1, 0, 1, 0, 1, 0, 1],
       [0, 1, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 1]], dtype=uint64)

In [149]:
feature_list = {"red":['A', 'B', 'C', 'K', 'N', 'P', 'R']
                             ,"black":['a', 'b', 'c', 'k', 'n', 'p', 'r']}
ind = 3
gc = convert_game_value(filelist[ind],feature_list,None)

In [170]:
x1,y1,val1 = gc.__next__()
x1 = np.transpose(x1,[1,2,0])
x1 = np.expand_dims(x1,axis=0)
oney = np.zeros(len(labels))
oney[label2ind[''.join(y1)]] = 1
x1.shape,oney.shape,val1

((1, 10, 9, 14), (2086,), 1)

In [171]:
np.sum(x1,axis=-1),''.join(y1)

(array([[[1, 1, 1, 1, 1, 1, 1, 1, 1],
         [0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 1, 0, 0, 0, 0, 0, 0],
         [1, 0, 0, 0, 1, 0, 1, 0, 1],
         [0, 0, 1, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0],
         [1, 0, 1, 0, 1, 0, 1, 0, 1],
         [1, 0, 0, 0, 1, 0, 0, 1, 0],
         [0, 0, 0, 0, 1, 0, 0, 0, 0],
         [0, 0, 1, 1, 1, 1, 1, 1, 1]]], dtype=uint64), 'a2b0')

In [106]:
sample_value[:10]

array([ 0,  1,  1,  1, -1,  0,  0,  1,  0,  0])

In [22]:
from collections import Counter
Counter(sample_value)

Counter({-1: 138, 0: 204, 1: 170})

In [56]:
len(labels)

2086

In [57]:
np.sum(sample_x1[0],axis=-1).shape

(10, 9)

In [58]:
!mkdir models

mkdir: 无法创建目录"models": 文件已存在


In [59]:
import os
if not os.path.exists("models/{}".format(model_name)):
    os.mkdir("models/{}".format(model_name))

In [60]:
N_BATCH = int(len(trainset) / BATCH_SIZE) * 75
#N_BATCH_TEST = 300 * (128 / BATCH_SIZE)


In [61]:
len(trainset)

2300

In [62]:
N_BATCH#,N_BATCH_TEST

300

In [112]:
latest_netname = NetMatainer(None,network_dir).get_latest()
latest_netname

'2018-05-30_20-00-13'

In [119]:
latest_netname = '2018-05-30_16-00-13'

In [120]:
from net.resnet import get_model

In [121]:
 (sess,graph),((X,training),(net_softmax,value_head,train_op_policy,train_op_value,policy_loss,accuracy_select,global_step,value_loss,nextmove,learning_rate,score)) = \
    get_model('data/prepare_weight/{}'.format(latest_netname),labels,GPU_CORE=GPU_CORE,FILTERS=128,NUM_RES_LAYERS=7,extra=True)

0
[None, 10, 9, 128]
[None, 10, 9, 128]
[None, 10, 9, 128]
[None, 10, 9, 128]
[None, 10, 9, 128]
[None, 10, 9, 128]
[None, 10, 9, 128]
inside res net [None, 10, 9, 128]
INFO:tensorflow:Restoring parameters from data/prepare_weight/2018-05-30_16-00-13


In [122]:
#with graph.as_default():
#    sess.run(tf.global_variables_initializer())

with graph.as_default():
    train_epoch = 58
    train_batch = 0
    saver = tf.train.Saver(var_list=tf.global_variables())
    saver.restore(sess,"models/{}/model_{}".format(model_name,train_epoch - 1))

In [123]:
train_epoch = 1
train_batch = 0

In [128]:
list(range(1,4))

[1, 2, 3]

In [127]:
restore = True
N_EPOCH = 3
DECAY_EPOCH = 20

class ExpVal:
    def __init__(self,exp_a=0.97):
        self.val = None
        self.exp_a = exp_a
    def update(self,newval):
        if self.val == None:
            self.val = newval
        else:
            self.val = self.exp_a * self.val + (1 - self.exp_a) * newval
    def getval(self):
        return round(self.val,2)
    
expacc_move = ExpVal()
exploss = ExpVal()
expsteploss = ExpVal()

begining_learning_rate = 1e-2

pred_image = None
if restore == False:
    train_epoch = 1
    train_batch = 0
for one_epoch in range(train_epoch,N_EPOCH):
    train_epoch = one_epoch
    pb = ProgressBar(worksum=N_BATCH * BATCH_SIZE,info=" epoch {} batch {}".format(train_epoch,train_batch))
    pb.startjob()
    
    for one_batch in range(N_BATCH):
        if restore == True and one_batch < train_batch:
            pb.auto_display = False
            pb.complete(BATCH_SIZE)
            pb.auto_display = True
            continue
        else:
            restore = False
        train_batch = one_batch
        
        batch_x,batch_y,batch_v = trainflow.next()['data']
        batch_v = np.expand_dims(np.nan_to_num(batch_v),1)
        # learning rate decay strategy
        batch_lr = begining_learning_rate * 2 ** -(one_epoch // DECAY_EPOCH)
        with graph.as_default():
            _,step_loss,step_acc_move,step_value = sess.run(
                [train_op_policy,policy_loss,accuracy_select,global_step],feed_dict={
                    X:batch_x,nextmove:batch_y,learning_rate:batch_lr,training:True,
                })
            _,step_value_loss,step_val_predict = sess.run(
                [train_op_value,value_loss,value_head],feed_dict={
                    X:batch_x,learning_rate:batch_lr,training:True,score:batch_v,
                })
            #batch_v = - batch_v
            #batch_x = np.concatenate((batch_x[:,::-1,:,7:],batch_x[:,::-1,:,:7]),axis=-1)
            #_,step_value_loss,step_val_predict = sess.run(
            #    [train_op_value,value_loss,value_head],feed_dict={
            #        X:batch_x,learning_rate:batch_lr,training:True,score:batch_v,
            #    })
            
        
        step_acc_move *= 100
        
        expacc_move.update(step_acc_move)
        exploss.update(step_loss)
        expsteploss.update(step_value_loss)

       
        pb.info = "EPOCH {} STEP {} LR {} ACC {} LOSS {} value_loss {}".format(
            one_epoch,one_batch,batch_lr,expacc_move.getval(),exploss.getval(),expsteploss.getval())
        
        pb.complete(BATCH_SIZE)
    print()
    with graph.as_default():
        saver = tf.train.Saver(var_list=tf.global_variables())
        saver.save(sess,"models/{}/model_{}".format(model_name,one_epoch))
    continue
    
    
    pb = ProgressBar(worksum=N_BATCH // 4 * BATCH_SIZE,info=" epoch {}".format(train_epoch))
    pb.startjob()
    losses = []
    value_losses = []
    accs = []
    for one_batch in range(N_BATCH // 4):
        batch_x,batch_y,batch_v = testflow.next()['data']
        batch_v = np.expand_dims(np.nan_to_num(batch_v),1)
        # learning rate decay strategy
        batch_lr = begining_learning_rate * 10 ** -(one_epoch // DECAY_EPOCH)
        with graph.as_default():
            step_loss,step_value_loss,step_acc_move,step_value = sess.run(
                [policy_loss,value_loss,accuracy_select,global_step],feed_dict={
                    X:batch_x,nextmove:batch_y,learning_rate:batch_lr,training:False,score:batch_v,
                })
        
        step_acc_move *= 100
        losses.append(step_loss)
        accs.append(step_acc_move)
        value_losses.append(step_value_loss)
        pb.complete(BATCH_SIZE)
    print("EPOCH {} valid loss {} value loss {} acc {}".format(train_epoch,np.average(losses)
                                                               ,np.average(value_losses),np.average(accs)))
    print()
    with graph.as_default():
        saver = tf.train.Saver(var_list=tf.global_variables())
        saver.save(sess,"models/{}/model_{}".format(model_name,one_epoch))

EPOCH 1 STEP 149 LR 0.01 ACC 0.07 LOSS 7.64 value_loss 0.39 50.00 % [=========================>-------------------------] 76800/153600 	 used:8s eta:8 s8/153600 	 used:0s eta:0 s

KeyboardInterrupt: 

In [157]:
step_val_predict

NameError: name 'step_val_predict' is not defined

In [129]:
batch_x.shape

(512, 10, 9, 14)

In [ ]:
"models/{}/model_{}".format(model_name,one_epoch)

In [55]:
!ls -l 'models/update_model/model_2.data-00000-of-00001'

-rw-rw-r-- 1 meta meta 38213816  5月 30 15:09 models/update_model/model_2.data-00000-of-00001


In [46]:
model_name

'update_model'

In [81]:
for f in ['data-00000-of-00001','meta','index']:
    src = "models/{}/model_{}.{}".format(model_name,one_epoch,f)
    dst = os.path.join(network_dir,"{}.{}".format(stamp,f))
    shutil.copyfile(src,dst)

In [16]:
sorted([i[:-6] for i in os.listdir('data/prepare_weight/') if '.index' in i])[::-1][:2]

['2018-05-24_21-19-55', '2018-05-23']

In [3]:
import os
new_name, old_name = sorted([i[:-6] for i in os.listdir('data/prepare_weight/') if '.index' in i])[::-1][:2]

In [4]:
new_name, old_name

('2018-05-30_18-41-03', '2018-05-30_16-00-13')